In [1]:
import pandas as pd
import numpy as np
import os
import sys
import Mixture
import multiprocessing
from joblib import Parallel, delayed
from random import seed
from random import sample 
from random import randint

In [2]:
lines = [2, 6]
cpu = multiprocessing.cpu_count()
rango = 1000

In [3]:
dataFrameSignature = pd.read_excel('data/Pre.proccessor.xlsx', sheet_name = 0) 

In [4]:
def createBetas (X, a, b, n):

    ns = randint(a, b)
    r = np.random.uniform(1, 0.2, ns)
    rn = r/r.sum()
    id = sample(range(n-1), ns)
    betas = np.repeat(.0, n, axis=0)
    for index, i in enumerate(id, start = 0):
        betas[i] = rn[index]

    vector = X.to_numpy(copy=True)    
    vector = vector.flatten()
        
    A = np.dot(X, betas) + vector[sample(range(len(vector)), len(X.index))]

    return betas, id, A

In [5]:
X = dataFrameSignature.iloc[:, 1:]

subjects = Parallel(n_jobs=cpu, backend='threading')(delayed(createBetas)(X = X, a = lines[0], b = lines[1], n = len(X.columns)) for i in range(rango))

# Getting expression matrix
vector = [x[2] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
Y2 = pd.DataFrame(np.column_stack(vector), columns=columns)
Y2.insert(0, 'Gene symbol', dataFrameSignature['Gene symbol'])

# Getting Real Betas Matrix
vector = [x[0] for x in subjects]
columns = ['V' + str(x+1) for x in range(len(subjects))]
betas = pd.DataFrame(np.column_stack(vector), columns=columns)

X = dataFrameSignature

vector = [len(x[1]) for x in subjects]
ids = pd.DataFrame(np.column_stack(vector))   

In [10]:
betas

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V991,V992,V993,V994,V995,V996,V997,V998,V999,V1000
0,0.000000,0.000000,0.000000,0.568245,0.443195,0.157061,0.363994,0.622493,0.239218,0.000000,...,0.401336,0.000000,0.000000,0.000000,0.159254,0.000000,0.214068,0.267057,0.000000,0.000000
1,0.128216,0.497463,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.278392,0.000000,0.000000,0.000000,0.094263,0.000000,0.116851,0.197989,0.127734,0.250682
2,0.199611,0.000000,0.265681,0.431755,0.556805,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.201331,0.288469,0.253885,0.000000,0.465207,0.141284,0.000000,0.312506,0.386324
3,0.000000,0.000000,0.303842,0.000000,0.000000,0.230515,0.000000,0.000000,0.068978,0.386895,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.127809,0.000000,0.165977,0.000000,0.258280
4,0.000000,0.502537,0.241619,0.000000,0.000000,0.174565,0.168698,0.000000,0.173278,0.000000,...,0.320272,0.261986,0.394635,0.000000,0.138729,0.000000,0.000000,0.081365,0.238593,0.000000
5,0.319977,0.000000,0.000000,0.000000,0.000000,0.253991,0.261244,0.377507,0.070414,0.000000,...,0.000000,0.000000,0.000000,0.267967,0.315290,0.106603,0.000000,0.124072,0.130059,0.104713
6,0.352196,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.208425,0.613105,...,0.000000,0.346109,0.316896,0.292617,0.000000,0.300381,0.331872,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.188858,0.000000,0.000000,0.183869,0.206064,0.000000,0.239689,0.000000,...,0.000000,0.190574,0.000000,0.185531,0.292464,0.000000,0.195925,0.163539,0.191108,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [ ]:
#Generate xlsx
with pd.ExcelWriter('betas.list.TIL9.xlsx') as writer:
    betas.to_excel(writer, sheet_name='beta')
    ids.to_excel(writer, sheet_name='id')
    Y2.to_excel(writer, sheet_name='A')
